## Rain prediction in Australia

#### Import required libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

import sklearn

from sklearn.svm import SVC
from sklearn.svm import SVR

from sklearn.impute import SimpleImputer

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score, recall_score, f1_score,ConfusionMatrixDisplay,classification_report
from sklearn.metrics import mean_squared_error

import optuna

import xgboost as xgb
from xgboost import XGBClassifier

<br>
<br>
<br>
<br>
<br>

#### Read dataset

In [2]:
dataframe_clean_wo_outl_wo_corr = pd.read_csv(r"C:\Users\Lucio\Documents\Github\Next-day-rain-prediction\1- Data\2- Processed\dataframe_clean_wo_outl_wo_corr.csv", index_col=0)
dataframe_clean_wo_outl_wo_corr.head()

,Location,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,RainToday,RainTomorrow
0,Albury,0.6,W,44.0,W,WNW,71.0,22.0,1007.7,0.0,0.0
1,Albury,0.0,WNW,44.0,NNW,WSW,44.0,25.0,1010.6,0.0,0.0
2,Albury,0.0,WSW,46.0,W,WSW,38.0,30.0,1007.6,0.0,0.0
3,Albury,0.0,NE,24.0,SE,E,45.0,16.0,1017.6,0.0,0.0
4,Albury,1.0,W,41.0,ENE,NW,82.0,33.0,1010.8,0.0,0.0


dataframe_clean_wo_outl_wo_corr characteristics:
- Removed univariated outliers
- Removed variables with high collinearity

<br>
<br>
<br>
<br>
<br>

#### Encode Categorical Features

In [3]:
dataframe_encoded = pd.get_dummies(dataframe_clean_wo_outl_wo_corr)
dataframe_encoded.head()

,Rainfall,WindGustSpeed,Humidity9am,Humidity3pm,Pressure9am,RainToday,RainTomorrow,Location_Adelaide,Location_Albany,Location_Albury,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,0.6,44.0,71.0,22.0,1007.7,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.0,44.0,44.0,25.0,1010.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.0,46.0,38.0,30.0,1007.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0.0,24.0,45.0,16.0,1017.6,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1.0,41.0,82.0,33.0,1010.8,0.0,0.0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


<br>
<br>
<br>
<br>
<br>

### Model Tranining

In [4]:
#Create X and y dataframes
X = dataframe_encoded[[c for c in dataframe_encoded if c != 'RainTomorrow']].values
y = dataframe_encoded[['RainTomorrow']]

In [5]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y,random_state=0, test_size=0.30)

In [33]:
'''
def objective_xgb(trial):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)

    param = {
        "verbosity": 1,
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        #"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'eval_metric': 'error',  # Use error (1 - accuracy) as the evaluation metric for binary classification
    }

    if param["booster"] == "gbtree":# or param["booster"] == "gblinear":
        param["subsample"] = trial.suggest_float("subsample", 1e-8, 1.0, log=True)
        param["n_estimators"] = trial.suggest_int("n_estimators", 1, 1000)        
        param["max_depth"] = trial.suggest_int("max_depth", 1, 64)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    '''
'''
    if param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 64)
        param["subsample"] = trial.suggest_float("subsample", 1e-8, 1.0, log=True)
        param["n_estimators"] = trial.suggest_int("n_estimators", 1, 1000) 
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    '''
'''
    bst = xgb.train(param, dtrain)
    y_pred = bst.predict(dvalid)
    y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]  # Convert probabilities to binary predictions
    
    accuracy = sklearn.metrics.accuracy_score(y_val, y_pred_binary)
    
    return -accuracy  #An objective value linked with the Trial object
'''

IndentationError: unexpected indent (1396311297.py, line 24)

In [16]:
def objective_xgb(trial):
    param = {
        "verbosity": 1,
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        'eval_metric': 'error',
    }

    param["booster"] == "gbtree"
    param["subsample"] = trial.suggest_float("subsample", 1e-8, 1.0, log=True)
    param["n_estimators"] = trial.suggest_int("n_estimators", 1, 1000)        
    param["max_depth"] = trial.suggest_int("max_depth", 1, 64)
    param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
    param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
    param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    bst = xgb.XGBClassifier(**param)
    bst.fit(X_train, y_train)

    y_pred = bst.predict(X_val)
    accuracy = sklearn.metrics.accuracy_score(y_val, y_pred)

    return -accuracy  #Negative accuracy to maximize it (because 'eval_metric': 'error')

In [7]:
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X_sc = sc_X.fit_transform(X)
y_sc = sc_y.fit_transform(y)

In [8]:
X_sc_train, X_sc_val, y_sc_train, y_sc_val = sklearn.model_selection.train_test_split(X_sc, y_sc, random_state=0, test_size=0.30)

In [9]:
study_xgb = optuna.create_study()
study_xgb.optimize(objective_xgb, n_trials=30)
study_xgb.best_params

[I 2023-11-21 20:15:32,382] A new study created in memory with name: no-name-ec17d3c8-25e4-46d1-9181-d2999fe2b870
[I 2023-11-21 20:15:35,207] Trial 0 finished with value: -0.7825638202711072 and parameters: {'booster': 'gbtree', 'lambda': 1.2193411486911568e-05, 'alpha': 0.0731932908182318, 'subsample': 3.84199150016924e-07, 'n_estimators': 60, 'max_depth': 62, 'eta': 1.182501136059506e-08, 'gamma': 6.3326899755224e-07, 'grow_policy': 'depthwise'}. Best is trial 0 with value: -0.7825638202711072.
[I 2023-11-21 20:16:09,241] Trial 1 finished with value: -0.7825638202711072 and parameters: {'booster': 'gbtree', 'lambda': 0.00045276207225820934, 'alpha': 0.060164958145063036, 'subsample': 3.461730651291235e-07, 'n_estimators': 829, 'max_depth': 55, 'eta': 0.007638531736002273, 'gamma': 6.276566457831419e-05, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -0.7825638202711072.
[I 2023-11-21 20:16:24,028] Trial 2 finished with value: -0.7825638202711072 and parameters: {'booster': 

KeyboardInterrupt: 

In [10]:
xgb_params = study_xgb.best_params
xgb_params

{'booster': 'gbtree',
 'lambda': 0.23289732485768289,
 'alpha': 1.4055112625430121e-08,
 'subsample': 0.9062248911722238,
 'n_estimators': 600,
 'max_depth': 23,
 'eta': 8.25493315801435e-05,
 'gamma': 0.5473902699194424,
 'grow_policy': 'lossguide'}

dtrain = xgb.DMatrix(X_sc_train, y_sc_train)
dtest = xgb.DMatrix(X_sc_val)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=50,
    verbose_eval=200, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()
plt.show()

In [11]:
model = XGBClassifier(**xgb_params, silent=0)

model.fit(X_sc_train, y_sc_train, eval_set=[(X_sc_val, y_sc_val)], early_stopping_rounds=10, verbose=True)

C:\Users\Lucio\miniconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[20:54:45] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-logloss:0.69310
[1]	validation_0-logloss:0.69306
[2]	validation_0-logloss:0.69301
[3]	validation_0-logloss:0.69296
[4]	validation_0-logloss:0.69292
[5]	validation_0-logloss:0.69287
[6]	validation_0-logloss:0.69283
[7]	validation_0-logloss:0.69278
[8]	validation_0-logloss:0.69273
[9]	validation_0-logloss:0.69269
[10]	validation_0-logloss:0.69264
[11]	validation_0-logloss:0.69260
[12]	validation_0-logloss:0.69255
[13]	validation_0-logloss:0.69250
[14]	validation_0-logloss:0.69246
[15]	validation_0-logloss:0.69241
[16]	validation_0-logloss:0.69237
[17]	validation_0

[226]	validation_0-logloss:0.68293
[227]	validation_0-logloss:0.68289
[228]	validation_0-logloss:0.68284
[229]	validation_0-logloss:0.68280
[230]	validation_0-logloss:0.68275
[231]	validation_0-logloss:0.68271
[232]	validation_0-logloss:0.68266
[233]	validation_0-logloss:0.68262
[234]	validation_0-logloss:0.68258
[235]	validation_0-logloss:0.68253
[236]	validation_0-logloss:0.68249
[237]	validation_0-logloss:0.68244
[238]	validation_0-logloss:0.68240
[239]	validation_0-logloss:0.68236
[240]	validation_0-logloss:0.68231
[241]	validation_0-logloss:0.68227
[242]	validation_0-logloss:0.68222
[243]	validation_0-logloss:0.68218
[244]	validation_0-logloss:0.68214
[245]	validation_0-logloss:0.68209
[246]	validation_0-logloss:0.68205
[247]	validation_0-logloss:0.68200
[248]	validation_0-logloss:0.68196
[249]	validation_0-logloss:0.68192
[250]	validation_0-logloss:0.68187
[251]	validation_0-logloss:0.68183
[252]	validation_0-logloss:0.68178
[253]	validation_0-logloss:0.68174
[254]	validation_0-l

[461]	validation_0-logloss:0.67274
[462]	validation_0-logloss:0.67270
[463]	validation_0-logloss:0.67266
[464]	validation_0-logloss:0.67262
[465]	validation_0-logloss:0.67257
[466]	validation_0-logloss:0.67253
[467]	validation_0-logloss:0.67249
[468]	validation_0-logloss:0.67245
[469]	validation_0-logloss:0.67240
[470]	validation_0-logloss:0.67236
[471]	validation_0-logloss:0.67232
[472]	validation_0-logloss:0.67228
[473]	validation_0-logloss:0.67223
[474]	validation_0-logloss:0.67219
[475]	validation_0-logloss:0.67215
[476]	validation_0-logloss:0.67211
[477]	validation_0-logloss:0.67206
[478]	validation_0-logloss:0.67202
[479]	validation_0-logloss:0.67198
[480]	validation_0-logloss:0.67194
[481]	validation_0-logloss:0.67189
[482]	validation_0-logloss:0.67185
[483]	validation_0-logloss:0.67181
[484]	validation_0-logloss:0.67177
[485]	validation_0-logloss:0.67172
[486]	validation_0-logloss:0.67168
[487]	validation_0-logloss:0.67164
[488]	validation_0-logloss:0.67160
[489]	validation_0-l

XGBClassifier(alpha=1.4055112625430121e-08, base_score=0.5, booster='gbtree',
              callbacks=None, colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=1, early_stopping_rounds=None,
              enable_categorical=False, eta=8.25493315801435e-05,
              eval_metric=None, gamma=0.5473902699194424, gpu_id=-1,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints='', lambda=0.23289732485768289,
              learning_rate=8.25493335e-05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=23, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=600,
              n_jobs=0, num_parallel_tree=1, predictor='auto', ...)

In [12]:
y_predicted = model.predict(X_sc_val)
y_predicted

array([0, 0, 0, ..., 1, 0, 0])

In [13]:
conf_matrix = confusion_matrix(y_sc_val, y_predicted)

accuracy = accuracy_score(y_sc_val, y_predicted)
precision = precision_score(y_sc_val, y_predicted)
recall = recall_score(y_sc_val, y_predicted)
f1 = f1_score(y_sc_val, y_predicted)
roc_auc = roc_auc_score(y_sc_val, y_predicted)

print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("ROC AUC:", roc_auc)

Confusion Matrix:
 [[32229  1890]
 [ 4874  4606]]
Accuracy: 0.8448588270373174
Precision: 0.709051724137931
Recall: 0.4858649789029536
F1-Score: 0.5766149223835754
ROC AUC: 0.715235311925758
